# QAA: Maximum Likelihooh algorithm for QPE

As explained in *04_AmplitudeAmplification_Problems.ipynb* for calculating $E_{x\sim p}(f)$ (expected value of a function $f(x)$ over a domain that follows a distribution probability $p(x)$) using a **Quantum Amplification Amplitude** procedure based on **Groover's algorithm** the estimation of the phase of the **Groover** operator is needed.

For this **Quantum Phase Estimation** (**QPE**) the inverse of the **Quantum Fourier Transform** (**QFT**) is needed.

Quantum circuit implementation of **QFT** are very complex and very long and deep so its use in actual quantum computers is noisy and not very useful. 

One aproximation in order to have a **QPE** wihtout **QFT** is using the Maximum Likelihood algorithm. In this Notebook this algorithm is reviewed. 

Additionally we will show how to use the implementation of this algorithm done in the maximum_likelihood_qpe.py script


In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys, os
%matplotlib inline
%load_ext qat.core.magic
#QPU connection
QLMASS = True
if QLMASS:
    try:
        from qat.qlmaas import QLMaaSConnection
        connection = QLMaaSConnection()
        LinAlg = connection.get_qpu("qat.qpus:LinAlg")
        lineal_qpu = LinAlg()
    except (ImportError, OSError) as e:
        print('Problem: usin PyLinalg')
        from qat.qpus import PyLinalg
        lineal_qpu = PyLinalg()
else:
    print('User Forces: PyLinalg')
    from qat.qpus import PyLinalg
    lineal_qpu = PyLinalg()    

## 1. Data Loading

First we need to discretized the probability $p(x)$ and the function $f(x)$ and load the arrays in a quantum state

In [ ]:
from QuantumMultiplexors_Module_examples import expectation_loading_data
from PhaseAmplification_Module import load_q_gate
from AuxiliarFunctions import  get_histogram, postprocess_results, test_bins, run_job

In [ ]:
def p(x):
    return x*x
def f(x):
    return np.sin(x)

In [ ]:
#number of Qbits for the circuit
n_qbits = 8
#The number of bins 
m_bins = 2**n_qbits
LowerLimit = 0.0
UpperLimit = 1.0 

X, p_X = get_histogram(p, LowerLimit, UpperLimit, m_bins)
f_X = f(X)

Next cell creates the quantum program for loading data and the corresponding **Groover** operator ($\mathcal{\hat{Q}}$) based on $\mathcal{P}$ and $\mathcal{R}$ gates

In [ ]:
Qprog, P_Gate, R_gate = expectation_loading_data(p_X, f_X)
Q_Gate = load_q_gate(P_Gate, R_gate)

In [ ]:
circuit = Qprog.to_circ()
%qatdisplay circuit

%qatdisplay Q_Gate --depth 1

## 2. ML-QPE algorithm step by step.

The main problem is the following: We have an **Groover** like operator, $\hat{Q}$ which is equivalent to a rotation around **y-axis** of a $\theta$ angle. This angle is unknow (a priori) and we want to compute it. We know that using **QPE** with **QFT** allows us get the angle but we know too that **QFT** is a complex and a prone error operation for a quantum computer. Usign the **Maximum Likelihood Qauntum Phase Estimation** (**ML-QPE**) we can obtain this $\theta$ without using a **QFT**.

The **MaximumLikelihoodQPE** class allow us implement this algorithm. In this section we are going to describe the class step by step and explain the basics of the **ML-QPE** algorithm

In [ ]:
#import the class
from maximum_likelihood_qpe import MaximumLikelihoodQPE

In [ ]:
#Instantitate the class with the program and the Q operator
ml_qpe = MaximumLikelihoodQPE(Qprog, Q_Gate)

With the method *apply_gate* we can apply the **Q_Gate** operator $m_k$ times to the initial quantum program **Qprog**, generate the correspondient circuit and job and execute it a desired number of shots (nbshots). 

The method returns a pandas dataframe with results and the generated circuit and jobs

In [ ]:
#number of times operator Q should be applied
m_k = 4
#Number of measurements of the last qbit will be done. 0 calculates the true probability
nbshots = 0
pdf, circuit, job = ml_qpe.apply_gate(m_k, nbshots)

In [ ]:
%qatdisplay circuit

The resulting pdf provides following info:

* **Probability_|0>**: Probability for obtaining state $|0\rangle$. If nbshots = 0 then is the true probabilitiy computed. Otherwise it is its frequency.
* **Probability_|1>**: Probability for obtaining state $|1\rangle$. If nbshots = 0 then is the true probabilitiy computed otherwise it is its frequency.
* **m_k**: number of operator $\hat{Q}$ was applied.
* **h_k**: nummber of times the state $|1\rangle$ were obtained. If nbshots = 0 then is computed by multiply 100*Probability_|1> (can be changed using the property: **default_nbshots**). Otherwise It is the propper number of times state $|1\rangle$  was obtained.
* **n_k**: number of measurements done. If nbshots = 0 then n_k=100 (can be changed using the property: **default_nbshots**). Otherwise it will be equal to nbshots


In [ ]:
pdf

**NOTE**

This method creates a deep copy of the *Qprog* object each time is called, so the original *Qprog* do not suffer any modification (neither the property q_prog of the ml_qpe object)

In [ ]:
circuit = Qprog.to_circ()
%qatdisplay circuit

In [ ]:
circuit = ml_qpe.q_prog.to_circ()
%qatdisplay circuit

With the info of the resulting pandas DataFrame (pdf) we can compute the asociated **Likelihood**. 

What is the **Likelihood**?

In this case we have applied the operator $\hat{Q}^{m_k}$. As we know this operator is equivalent to a rotation around the y axis of $(2m_k+1)\theta$. Finally we have done several measurements of the last qbit and we have obtained some statistics (that are stored in pdf). 
In this case the **Likelihood** is the probability of obtaining the given measurements conditioned to a fixed angle $\theta$. In the case of our **Groover** operator the **Likelihood** for $m_k$ measurements of the state $|1\rangle$ when $n_k$ measurements were done will be:

$$l_{k}(h_k/\theta) = (\sin^2[(2*m_k+1)\theta])^{h_k}(\cos^2[(2*m_k+1)\theta])^{n_k-h_k}$$

This is because the probability of the state $|1\rangle$ is given by $\sin^2[(2*m_k+1)\theta]$ the probability of the state $|0\rangle$ is given by: $\cos^2[(2*m_k+1)\theta]$ and each measurment is independent of the other measurements (so a binomial distribution can be used)

For computing purpouses, usually, instead of the **Likelihood** the minus logarithm of the **Likelihood** is provided:

$$-\ln{l_{k}(h_k/\theta)} = -2h_k\ln(\sin[(2*m_k+1)\theta])-2(N_k-h_k)\ln(\cos[(2*m_k+1)\theta])$$

For computing the logarithm of the **Likelihood** we use the method **launch_likelihood** that needs the computed pdf and N that is the number of divisions of the domain (basically we are going to compute the log of **Likelihood** of the measurements of the pdf for several angles. N is the number of angles we are going to use). The output of the method is a new pdf with different $\theta$ angles and the correspondent log **Likelihood** for getting the given measurements of the pdf


In [ ]:
likelihood_ = ml_qpe.launch_likelihood(pdf, N=100)

We can plot the **Likelihood** with respect the posible $\theta$.

In [ ]:
plt.plot(likelihood_['theta'], likelihood_['l_k'], '-o')
plt.xlabel('theta')
plt.ylabel('Likelihood')

As can be seen there are some values of $\theta$ where the **-log of Likelihood** presents a minimums. We expect that the $\theta$ we are looking for would be one of these values 

## 3. ML-QPE algorithm 

With the before section we can know complete the **ML-QPE** algorithm:

1. Select a list  of different $m_k$ applications of the **Groover** like operator (one posible option will be for example:  $k=0,1,2,3...$ $m_k=1,4,8,16...$).
2. For each $m_k$ apply the **Groover** like operator and meas the last qbit of the circuit a fixed number of times ($n_k$) and get the number of state $|1\rangle$ measurements ($h_k$).
3. For eack $m_k$ the associated **Likelihood** will be:
$$l_{k}(h_k/\theta) = (\sin^2[(2*m_k+1)\theta])^{h_k}(\cos^2[(2*m_k+1)\theta])^{n_k-h_k}$$
4. So for each $m_k$ we have a $n_k$ and a $h_k$ and a associated **Likelihood** $l_k$. So we can compute the final **Likelihood** as:
$$L(\mathbf{h}/\theta) = \prod_{k=0}^{M}{l_{k}(h_k/\theta)}$$
$$\mathbf{h} = (h_0, h_1,...,h_M)$$
5. The idea is find the $\theta_{a}$ that maximizes the total **Likelihood**
$$\theta_{a} = arg \ max {L(\mathbf{h}/\theta)}$$ 

Maximizing **Likelihood** is equivalent to minimize the **- the logarithm of the Likelihood**. Usually this is preferred over the former so we are going to:


$$\theta_{a} = arg \ min \sum_{k=0}^{M} \Big( -2h_k\ln(\sin[(2*m_k+1)\theta])-2(N_k-h_k)\ln(\cos[(2*m_k+1)\theta]) \Big)$$

We can implement this loop manually:

In [ ]:
nbshots = 0
#select a list of m_k's
list_of_mks = [1, 2, 4, 8, 16, 32]
list_of_pdfs = []
list_of_circuits = []
for m_k in list_of_mks:
    pdf, circuit, job = ml_qpe.apply_gate(m_k, nbshots)
    list_of_pdfs.append(pdf)
    list_of_circuits.append(circuit)
pdf_final = pd.concat(list_of_pdfs)
pdf_final.reset_index(drop=True, inplace=True)

For all the $m_k$ used we generated a pandas DataFrame (pdf_final) with the complete information of the results:

In [ ]:
pdf_final

We can use again the **launch_likelihood** method for computing the **-log(likelihood)** for several $\theta$'s and plot it:

In [ ]:
final_likelihood = ml_qpe.launch_likelihood(pdf_final, 100)

In [ ]:
plt.plot(final_likelihood['theta'], final_likelihood['l_k'], '-o')
plt.xlabel('theta')
plt.ylabel('-Log(Likelihood)')
#plt.ylim(0, 500)

For calculating the $\theta_a$ we can use different methods. In general we can create a **likelihood** python function (there is on created in the iterative_quantum_pe.py called *likelihood*) and provide the information of the **pdf_final** to a software optimization library that minimizes it.

For example we can use the **scipy.optimize** package for doing such minimization:

In [ ]:
#-log(likelihood) function
from maximum_likelihood_qpe import likelihood
#optimizers from scipy
import scipy.optimize as so 

There is a *minimize* generic method in **scipy.optimize** that allow us minimize a given function:

In [ ]:
#initial theta value
theta_0 = [0.9]
sol = so.minimize(
    likelihood, 
    theta_0,
    (pdf_final['m_k'], pdf_final['h_k'], pdf_final['n_k']), #arguments needed for likelihood function
    method = 'Nelder-Mead',
    #method ='Powell',
    #1000,
    #disp=True,
    options = {'maxiter': 1000, 'disp': True},# 'xtol':1e-7}
)

print(sol)
print('Theta optimum :{}'.format(sol.x))

In general this kind of methods have a very high dependency of the initial guess and do not work properly (change the theta_0 for see different solutions).

In this case a working robust method is the *brute* method that uses brute force for getting the minimum value. In this case the computational cost is not very expensive and can be used without any problem.

This method need:

* function to minimize,
* the range for the variable to optimize (in this case $\theta \in [0, \frac{\pi}{2}]$)
* the $m_k$, $h_k$ and the $n_k$ from the **pdf_final**

We can give other parameters that control the optimization procces

In [ ]:
#Domain for searching
theta_domain = [1e-09, 0.5*np.pi-1e-09]
solution = so.brute(
    likelihood, 
    [theta_domain],
    (pdf_final['m_k'], pdf_final['h_k'], pdf_final['n_k']),
    1000,
    disp=True,
    #options = {'maxiter': 100, 'disp': True}
)

In [ ]:
print('ŧheta optimum: {}'.format(solution))

In [ ]:
print('Optimum theta : {}'.format(solution[0]))
print('Theoric theta: {}'.format(np.arcsin(sum(f_X*p_X)**0.5)))

As can be seen an aceptable estimation of the $\theta$ angle is porovided

The **MaximumLikelihoodQPE** class implements this optimizer into the method **launch_optimizer** where the final_pdf should be provided!

In [ ]:
theta = ml_qpe.launch_optimizer(pdf_final)
print('Optimum theta: {}'.format(theta))

## 4. ML-QPE algorithm using MaximumLikelihoodQPE class

The **MaximumLikelihoodQPE** class provide a **run** method for doing a complete **ML-QPE** algorithm. In this section we explain how to use it.

### 4.1 Instantiate the class

First we need to instantiate the class. Following argumens can be provided:

* q_prog : QLM quantum program (mandatory)
    * Quantum program where the Groover-like operator will be applied
* q_gate : QLM gate (mandatory)
    * QLM gate that implements the Groover-like operator
* kwars : dictionary
    dictionary that allows the configuration of the ML-QPE algorithm:
    Implemented keys:
    * list_of_mks : list
        * python list with the different m_ks for executing the algortihm
    * qpu : QLM solver
        * solver for simulating the resulting circutis
    * delta : float 
        * For avoiding problems when calculating the domain for theta
    * default_nbshots : int
        * default number of measurements for computing freqcuencies when nbshots for quantum job is 0
    * iterations : int
        * number of iterations of the optimizer
    * display : bool
        * for displaying additional information in the optimization step
    * nbshots : int
        * number of shots for quantum job. If 0 exact probabilities will be computed. 


In [ ]:
from maximum_likelihood_qpe import MaximumLikelihoodQPE

In [ ]:
arg_dictionary = {
    'list_of_mks': range(15),
    'qpu': lineal_qpu,
    'delta': 1e-3,
    'default_nbshots' : 100,
    'iterations' : 100,
    'display' :  True,
    'nbshots' : 0
    
}

In [ ]:
ml_qpe = MaximumLikelihoodQPE(Qprog, Q_Gate, **arg_dictionary)

In [ ]:
print('list_of_mks: {}'.format(ml_qpe.list_of_mks))
print('delta: {}'.format(ml_qpe.delta))
print('default_nbshots: {}'.format(ml_qpe.default_nbshots))
print('iterations: {}'.format(ml_qpe.iterations))
print('display: {}'.format(ml_qpe.disp))
print('nbshots: {}'.format(ml_qpe.nbshots))

### 4.2 Execute run method

The run method execute a complete ML-QPE algorithm based on the parameters you passed to the class when you call it. If some of the parameters were not passes default ones will be used. To the **run** method you can provide:
* list_of_mks
* nbshots

The corresponding properties will be overwrite with the new ones

In [ ]:
ml_qpe.run()

When the **run** method finishes different information were stored in different porperties. Most important ones:

* theta: is the phase estimation for the Groover-like operator
* pdf_mks: pandas DataFrame with the results of the measurement for the different m_k's
* list_of_circuits: pyhton list with all the quantum circuits created for executing the algorithm

In [ ]:
print('The phase for the operator is: {}'.format(ml_qpe.theta))

In [ ]:
ml_qpe.pdf_mks

In [ ]:
circ = ml_qpe.list_of_circuits[-1]
%qatdisplay circ

Additionally we can plot the final **Likelihood** of the obtained measurements invoking the **launch_likelihood** and given the *pdf_mks* property

In [ ]:
pdf_like = ml_qpe.launch_likelihood(ml_qpe.pdf_mks)

In [ ]:
plt.plot(pdf_like['theta'], pdf_like['l_k'], '-o')
plt.xlabel('theta')
plt.ylabel('-Log(Likelihood)')
#plt.ylim(0, 500)

 The strength of the ML-QPE algorithm is in the application of different $m_k$ and try to compute the total **Likelihood**. This can be in the following cells where we apply the launch_optimizer to an increasing number of $m_k$ (first on $m_k$ is selected, then 2 $m_k$ is selected and so on until use all the $m_k$'s for computing likelihood and the correspondiente theta).

In [ ]:
list_of_thetas = []
for i in range(1, len(ml_qpe.pdf_mks)):
    step = ml_qpe.pdf_mks[:i]
    step_theta = ml_qpe.launch_optimizer(step)
    list_of_thetas.append(step_theta)

In [ ]:
plt.plot(list_of_thetas, '-o')
plt.xlabel('number of m_k for optimization')
plt.ylabel('theta')

As can be seen when more $m_k$'s are used more stable is the obtained $\theta$. Additionally increasing the  number of $m_k$'s decreases the error estimation of the $\theta$

In [ ]:
theoric_theta = np.arcsin(sum(f_X*p_X)**0.5)
print('theoric_theta: {}'.format(theoric_theta))
AbsolutError = [abs(theta-theoric_theta) for theta in list_of_thetas]

plt.plot(AbsolutError, 'o')
plt.semilogy()